In [1]:
import snorkel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import labeling_function
from snorkel.preprocess import preprocessor

import pandas as pd

In [2]:
# Import the data set
crowd1 = pd.read_csv('crowd_round1_clean.csv')
crowd1 = crowd1[['DB_ID', 'DO_ID', 'disease', 'Drug_name', 'Context', 'relation']]

crowd2 = pd.read_csv('crowd_round2_clean.csv')
crowd2 = crowd2[['DB_ID', 'DO_ID', 'disease', 'Drug_name', 'Context', 'relation']]


crowd_df = crowd1

df_test_1 = pd.read_csv('experts_curated_clean.csv')
df_test_1 = df_test_1[['drug_id','do_id','disease_name', 'drug_name', 'context','Final']]
df_test_1 = df_test_1.rename(columns={"drug_id": "DB_ID", "do_id": "DO_ID","disease_name": "disease", "drug_name": "Drug_name", "context":"Context", "Final":"relation" })


In [4]:
df_train = crowd_df

In [878]:
#Split the test set in : test and validation set
y = df_test_1['relation'] 
X = df_test_1.drop(['relation'], axis=1)

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.5, random_state=0)

a,b = sss.split(X, y)

train_index = a[0]
dev_index = a[1]

df_test = df_test_1.iloc[train_index]
df_dev = df_test_1.iloc[dev_index]

In [883]:
# Encode the classes

Y_train = df_train.relation
Y_test = df_test.relation
Y_dev = df_dev.relation

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y_train)
le.classes_
labels_en_train = le.transform(Y_train) 
labels_en_test = le.transform(Y_test) 
labels_en_dev = le.transform(Y_dev) 
# Check the encoding
zero = list(le.inverse_transform([0]))
one = list(le.inverse_transform([1]))
two = list(le.inverse_transform([2]))
three = list(le.inverse_transform([3]))

print(zero, 'is encoded as 0')
print(one, 'is encoded as 1')
print(two, 'is encoded as 2')
print(three, 'is encoded as 3')

Y_train = labels_en_train
Y_test = labels_en_test 
Y_dev = labels_en_dev

['contraindication'] is encoded as 0
['effect'] is encoded as 1
['relief'] is encoded as 2
['treatment'] is encoded as 3


In [5]:
RELATION_CLASSES = {
'contraindication': 0,
'effect': 1,
'relief': 2,
'treatment':3 
}

ABSTAIN = -1

In [6]:
my_phrases = { 'effect': 
["hypersensitivity reactions",
"hypersensitivity reaction",
"associated with the risk of",
"to the risk of",
"a high risk for",
"a high risk of",
"high incidence of", 
"higher incidence of", 
" cause ",
" causes ",
"symptoms occure",
"teratogenic",
"site reaction",
"the risk of development",
"is associated with a risk of",
"symptoms of the poisoning",
"symptoms of poisoning",
"is the result of",
"might prove harmful"
],

 'contraindication': 
["not administrated to",
"is contraindicated", 
"contraindicated in",
"contraindicatedin",
"contraindicated with",
"should not be used",
"is contraindication for",
"is contraindication when",
"is contraindicated when",
"is contraindicated for",
"must not be used for",
"do not administer",
"should not initiate",
"not be administered to",
"do not initiate patients",
"contraindication for",
"should not be given",
"do not use",
"patients with a history of",
"history of", 
"adverse reactions",
"that have the following conditions",
"are not candidates for", 
"contraindication",
"allergy or hypersensitivity",
"are contraindicated",
"should not be employed",
"contraindicatedin"],
              
 'relief': 
["relief of the signs",
"relief of the signs and symptoms of",
"relief of signs",
"relief of symptoms",
"relief of the symptoms",
"help",
"helps",
"relief of signs and symptoms of",
"reduction of symptoms of",
"treatment of the symptoms of",
"for the relief",
"management of the signs and symptoms of"], 

 'treatment':
[" indicated for the treatment of",
"reduce the development of",
" indicated for the management of",
"for the management of",
"management of",
" indicated for the maintenance of remission", 
"or the treatment of",
"in the treatment of",
" indicated as",
" indicated in",
"be effective",
"active treatment of",
" indicated for",
"treatment of",
" indicated as an adjunct",
" indicated for use in the treatment of", 
" indicated for the intermittent treatment", 
" indicated to reduce the rate of",
" indicated for the rapid control",
" indicated for the control",
"reduce the risk of",
" indicated as adjunctive treatment",
"for the treatment of",
" indicated as an adjunct",
"areindicatedas",
"treatment is indicated",
"is indicatedin",
"indicated to improve",
"healing of ",
"supplemental therapy"]
           }

phrases = pd.DataFrame({ key:pd.Series(value) for key, value in my_phrases.items() })

effect = { 'phrases': 
["hypersensitivity reactions",
"hypersensitivity reaction",
"associated with the risk of",
"to the risk of",
"a high risk for",
"a high risk of",
"high incidence of", 
"higher incidence of", 
" cause ",
" causes ",
"symptoms occure",
"teratogenic",
"site reaction",
"the risk of development",
"is associated with a risk of",
"symptoms of the poisoning",
"symptoms of poisoning",
"is the result of",
"might prove harmful"
]}
effect_df = pd.DataFrame(effect)

 
contraindication = {'phrases': 
["not administrated to",
"is contraindicated", 
"contraindicated in",
"contraindicatedin",
"contraindicated with",
"should not be used",
"is contraindication for",
"is contraindication when",
"is contraindicated when",
"is contraindicated for",
"must not be used for",
"do not administer",
"should not initiate",
"not be administered to",
"do not initiate patients",
"contraindication for",
"should not be given",
"do not use",
"patients with a history of",
"history of", 
"adverse reactions",
"that have the following conditions",
"are not candidates for", 
"contraindication",
"allergy or hypersensitivity",
"are contraindicated",
"should not be employed",
"contraindicatedin"]}
contraindication_df = pd.DataFrame(contraindication)
              
relief = {'phrases': 
["relief of the signs",
"relief of the signs and symptoms of",
"relief of signs",
"relief of symptoms",
"relief of the symptoms",
"help",
"helps",
"relief of signs and symptoms of",
"reduction of symptoms of",
"treatment of the symptoms of",
"for the relief",
"management of the signs and symptoms of"]} 
relief_df = pd.DataFrame(relief)

treatment = {'phrases':
[" indicated for the treatment of",
"reduce the development of",
" indicated for the management of",
"for the management of",
"management of",
" indicated for the maintenance of remission", 
"or the treatment of",
"in the treatment of",
" indicated as",
" indicated in",
"be effective",
"active treatment of",
" indicated for",
"treatment of",
" indicated as an adjunct",
" indicated for use in the treatment of", 
" indicated for the intermittent treatment", 
" indicated to reduce the rate of",
" indicated for the rapid control",
" indicated for the control",
"reduce the risk of",
" indicated as adjunctive treatment",
"for the treatment of",
" indicated as an adjunct",
"areindicatedas",
"treatment is indicated",
"is indicatedin",
"indicated to improve",
"healing of ",
"supplemental therapy"]}
treatment_df = pd.DataFrame(treatment)

In [7]:
def check(x, string):
    return x in string


def find_phrases(text):
    # Find which phrases appear in the sentence
    class_effect = effect_df['phrases'].apply(lambda num : check(num,text))
    class_contraindication = contraindication_df['phrases'].apply(lambda num : check(num,text))
    class_relief = relief_df['phrases'].apply(lambda num : check(num,text))
    class_treatment = treatment_df['phrases'].apply(lambda num : check(num,text))
    
    #Create a dataset from them
    classes = {
    'effect': class_effect,
    'contraindication': class_contraindication,
    'relief': class_relief,
    'treatment':class_treatment 
    }

    classes = pd.DataFrame(classes)
    
    a = class_effect.index[class_effect == True].tolist()
    b = class_contraindication.index[class_contraindication == True].tolist()
    c = class_relief.index[class_relief == True].tolist()
    d = class_treatment.index[class_treatment == True].tolist()
    return classes, a, b, c, d

# return the position of a phrase/word in a sentence
def pos_of_phrase(string, substring):
    substring_length = len(substring)    
    def recurse(locations_found, start):
        location = string.find(substring, start)
        if location != -1:
            return recurse(locations_found + [location], location+substring_length)
        else:
            return locations_found

    return recurse([], 0)

# return the minimum distance of all phrase/word from the target disease
def get_diff_of_pos(classes, column, pos, string):
    name = classes.columns[column]
    my_col = classes[name]
    x = [k for k,v in enumerate(my_col) if v == True]
    
    candidate = []
    distances = []
    for i in range(len(x)):
        word = phrases[classes.columns[column]][x[i]]
        
        for i in range(len(pos_of_phrase(string, word))):
            distances.append( abs(pos -   pos_of_phrase(string, word)[i]) )

    return min(distances)


In [8]:
#Define the labelling functions

@labeling_function()
def phrases_claas(x):
    
    text = x.Context

    classes, a, b, c, d =find_phrases(text)


    # Check if you have unique class and label based on that 
    if len(a)!=0 and len(b)==len(c)==len(d)==0:
        #print('effect')
        return RELATION_CLASSES['effect']
    elif len(b)!=0 and len(a)==len(c)==len(d)==0:
        #print('contraindication')
        return RELATION_CLASSES['contraindication']
    elif len(c)!=0 and len(a)==len(b)==len(d)==0:
        #print('relief')
        return  RELATION_CLASSES['relief']
    elif len(d)!=0 and len(a)==len(b)==len(c)==0:
        #print('treatment')
        return  RELATION_CLASSES['treatment']
    elif len(a)==len(b)==len(c)==len(d)==0:
        #print('IDK')
        return ABSTAIN
    return ABSTAIN

@labeling_function()
def phrase_closeness(x):
    text = x.Context
    pos1 = text.find(x.disease)
    dis_len = len(x.disease)
    classes, a, b, c, d = find_phrases(text)
    
    minDist = len(text.split())
    category = ABSTAIN
    for class_,found_phrases in {'effect':a, 'contraindication':b, 'relief':c, 'treatment':d}.items():
        for phrase_index in found_phrases:
            phrase =phrases.loc[phrase_index,class_]
            pos2 = text.find(phrase)
            phr_len = len(phrase)
            if pos1 > pos2:
                span = text[pos2+phr_len:pos1]
            else:
                span = text[pos1+dis_len:pos2]
            words = span.split()
            distance = len(words)
            if minDist > distance:
                minDist = distance
                category = class_
    
    if category == ABSTAIN:
        return ABSTAIN
    return RELATION_CLASSES[category]

@labeling_function()
def crowd(x):
    #print (x.relation_type)
    if x.relation in RELATION_CLASSES:
        return RELATION_CLASSES[x.relation]
    return ABSTAIN

In [9]:
df_test.relation.unique()

NameError: name 'df_test' is not defined

In [891]:
lfs = [crowd, phrase_closeness, phrases_claas]
applier = PandasLFApplier(lfs)


In [892]:
L_train = applier.apply(train_df)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel




  0%|          | 0/2018 [00:00<?, ?it/s]



  1%|          | 18/2018 [00:00<00:11, 174.64it/s]



  2%|▏         | 36/2018 [00:00<00:11, 175.73it/s]



  3%|▎         | 56/2018 [00:00<00:10, 179.88it/s]



  4%|▍         | 76/2018 [00:00<00:10, 183.73it/s]



  5%|▍         | 96/2018 [00:00<00:10, 187.30it/s]



  6%|▌         | 116/2018 [00:00<00:10, 189.02it/s]



  7%|▋         | 136/2018 [00:00<00:09, 190.29it/s]



  8%|▊         | 156/2018 [00:00<00:09, 191.21it/s]



  9%|▊         | 176/2018 [00:00<00:09, 192.40it/s]



 10%|▉         | 196/2018 [00:01<00:09, 192.45it/s]



 11%|█         | 216/2018 [00:01<00:09, 194.17it/s]



 12%|█▏        | 236/2018 [00:01<00:09, 193.39it/s]



 13%|█▎        | 256/2018 [00:01<00:0

In [894]:
L_test= applier.apply(test_df)





  0%|          | 0/358 [00:00<?, ?it/s]



  5%|▍         | 17/358 [00:00<00:02, 166.09it/s]



 10%|▉         | 35/358 [00:00<00:01, 168.96it/s]



 15%|█▌        | 54/358 [00:00<00:01, 174.38it/s]



 21%|██        | 74/358 [00:00<00:01, 178.92it/s]



 26%|██▋       | 94/358 [00:00<00:01, 183.88it/s]



 32%|███▏      | 115/358 [00:00<00:01, 188.66it/s]



 38%|███▊      | 135/358 [00:00<00:01, 189.78it/s]



 43%|████▎     | 155/358 [00:00<00:01, 191.71it/s]



 49%|████▉     | 175/358 [00:00<00:00, 191.93it/s]



 54%|█████▍    | 194/358 [00:01<00:00, 188.89it/s]



 59%|█████▉    | 213/358 [00:01<00:00, 185.78it/s]



 65%|██████▍   | 232/358 [00:01<00:00, 183.65it/s]



 70%|███████   | 251/358 [00:01<00:00, 184.21it/s]



 76%|███████▌  | 271/358 [00:01<00:00, 186.48it/s]



 81%|████████  | 290/358 [00:01<00:00, 187.17it/s]



 86%|████████▋ | 309/358 [00:01<00:00, 186.01it/s]



 92%|█████████▏| 328/358 [00:01<00:00, 184.51it/s]



100%|██████████| 358/358 [00:01<00:00, 1

In [895]:
from snorkel.labeling import LabelModel
# Train LabelModel.
label_model = LabelModel(cardinality=4, verbose=True)
label_model.fit(L_train, n_epochs=200, seed=123, log_freq=20, l2=0.1, lr=0.01)

In [896]:
from snorkel.analysis import metric_score

preds_dev = label_model.predict(L_test)

In [897]:
preds_dev

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 3, 2, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 3, 1, 3, 1, 1, 3, 3, 1, 3, 1, 3, 3,

In [898]:
acc = metric_score(Y_test, preds_dev, probs=None, metric="accuracy")

ValueError: Found input variables with inconsistent numbers of samples: [179, 358]

In [869]:
acc

0.8603351955307262

In [871]:
LFAnalysis(L_dev, lfs).lf_summary(Y_dev).head()

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
crowd,0,"[0, 1, 2, 3]",1.000000,0.983240,0.245810,179,0,1.000000
phrase_closeness,1,"[0, 1, 2, 3]",0.983240,0.983240,0.245810,132,44,0.750000
phrases_claas,2,"[0, 1, 2, 3]",0.553073,0.553073,0.089385,83,16,0.838384
